### Import packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import cv2
#from pydicom import dcmread
#from pydicom.data import get_testdata_file
#from PIL import Image

#from tensorflow.keras import Sequential, layers, models, optimizers
#from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.layers.experimental.preprocessing import Rescaling ?

## Création du dataset

### Import dataset créé à partir des fichiers .csv

In [3]:
from covid_project.data import merged_csv_df, csv_simple_df, load_image_data
csv_df = merged_csv_df('local_path')
csv_df.head(3)

,boxes,label,StudyInstanceUID,image_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,000a312787f2,0,1,0,0
1,NaN,none 1 0 0 1 1,ff0879eb20ed,000c3a3f293f,1,0,0,0
2,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0012ff7358bc,0,1,0,0


In [4]:
simple_df = csv_simple_df('local_path')
simple_df.head(3)

,StudyInstanceUID,image_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,5776db0cec75,000a312787f2,0,1,0,0
1,ff0879eb20ed,000c3a3f293f,1,0,0,0
2,9d514ce429a7,0012ff7358bc,0,1,0,0


### Création dataset image

Utilisation d'un code OU de l'autre, les codes se recoupent

#### Code Claire

In [ ]:
# Si sur Colab, exécuter cette cellule
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# cellule à exécuter sur colab
image_data = load_image_data('colab')
image_data[0]

In [5]:
# OU
# cellule à exécuter en local
image_data = load_image_data('direct')
image_data[0]

['65761e66de9f',
 '00086460a852',
 'F',
 [0.148, 0.148],
 'MONOCHROME2',
 2330,
 2783,
 array([[1392, 1389, 1390, ...,    0,    0,    0],
        [1377, 1411, 1370, ...,    0,    0,    0],
        [1373, 1397, 1401, ...,    0,    0,    0],
        ...,
        [   0,    0,    0, ..., 2476, 2490, 2453],
        [   0,    0,    0, ..., 2515, 2541, 2509],
        [   0,    0,    0, ..., 2517, 2489, 2488]], dtype=uint16)]

In [6]:
train_metadata = pd.DataFrame(image_data, columns=['image_id','image_StudyInstanceUID','Patient_Sex', "ImagerPixelSpacing", "PhotometricInterpretation", 'image_Rows','image_Columns','pixel_array'])
print(train_metadata.shape)
train_metadata.head()

(5, 8)


,image_id,image_StudyInstanceUID,Patient_Sex,ImagerPixelSpacing,PhotometricInterpretation,image_Rows,image_Columns,pixel_array
0,65761e66de9f,00086460a852,F,"[0.148, 0.148]",MONOCHROME2,2330,2783,"[[1392, 1389, 1390, 1390, 1350, 1385, 1360, 13..."
1,51759b5579bc,000c9c05fd14,M,"[0.125, 0.125]",MONOCHROME2,3093,2850,"[[82, 81, 80, 81, 79, 80, 81, 78, 79, 77, 76, ..."
2,bb4b1da810f3,0051d9b12e72,F,"[0.15, 0.15]",MONOCHROME1,2336,2836,"[[24335, 24448, 24548, 24551, 24336, 24244, 24..."
3,3019399c31f4,005057b3f880,F,"[0.139, 0.139]",MONOCHROME1,3052,3052,"[[2848, 2863, 2885, 2891, 2899, 2919, 2894, 28..."
4,f6293b1c49e2,00292f8c37bd,M,"[0.139, 0.139]",MONOCHROME2,2539,3050,"[[86, 88, 86, 86, 87, 86, 87, 87, 87, 87, 87, ..."


In [7]:
print(type(train_metadata['pixel_array'][0]))
print(train_metadata['pixel_array'][0].shape)
train_metadata['pixel_array'][0]

<class 'numpy.ndarray'>
(2330, 2783)


array([[1392, 1389, 1390, ...,    0,    0,    0],
       [1377, 1411, 1370, ...,    0,    0,    0],
       [1373, 1397, 1401, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ..., 2476, 2490, 2453],
       [   0,    0,    0, ..., 2515, 2541, 2509],
       [   0,    0,    0, ..., 2517, 2489, 2488]], dtype=uint16)

#### Code Zeinab

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#éxecuter cette cellule sur google drive
destination_path = '/content/drive/MyDrive/New_train_subset/'

In [ ]:
# éxecuter cette cellule si en local
destination_path =  "/mnt/c/Users/Administrator/Desktop/COVID_Data/train_subset/New_train_subset" 

In [ ]:
images_list = os.listdir(destination_path)

In [ ]:
columns = ["PatientSex", "ImagerPixelSpacing",
           "SeriesInstanceUID", "PhotometricInterpretation",
           "Rows", "Columns", "pixel_array"]

In [ ]:
train_metadata =  pd.DataFrame(columns=columns)
new_images_list =[]
for i in images_list:
    fs=pydicom.read_file(destination_path+"/"+i, stop_before_pixels=True)
    train_metadata = train_metadata.append({field: fs[field].value for field in columns}, ignore_index=True)
    element = i.replace(".dcm", "")
    new_images_list.append(element)
train_metadata['image_id'] = new_images_list

In [ ]:
print(train_metadata.shape)
train_metadata.head()

### Merge des df

In [15]:
big_df = train_metadata.merge(csv_df, on = 'image_id', how='inner')
if big_df['image_StudyInstanceUID'].all() == big_df['StudyInstanceUID'].all():
    big_df = big_df.drop(columns= 'StudyInstanceUID')
big_df.head()

,image_id,image_StudyInstanceUID,Patient_Sex,ImagerPixelSpacing,PhotometricInterpretation,image_Rows,image_Columns,pixel_array,boxes,label,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,65761e66de9f,00086460a852,F,"[0.148, 0.148]",MONOCHROME2,2330,2783,"[[1392, 1389, 1390, 1390, 1350, 1385, 1360, 13...","[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,0,1,0,0
1,51759b5579bc,000c9c05fd14,M,"[0.125, 0.125]",MONOCHROME2,3093,2850,"[[82, 81, 80, 81, 79, 80, 81, 78, 79, 77, 76, ...",NaN,none 1 0 0 1 1,0,0,0,1
2,bb4b1da810f3,0051d9b12e72,F,"[0.15, 0.15]",MONOCHROME1,2336,2836,"[[24335, 24448, 24548, 24551, 24336, 24244, 24...","[{'x': 812.54698, 'y': 1376.41291, 'width': 62...",opacity 1 812.54698 1376.41291 1435.14793 1806...,0,0,0,1
3,3019399c31f4,005057b3f880,F,"[0.139, 0.139]",MONOCHROME1,3052,3052,"[[2848, 2863, 2885, 2891, 2899, 2919, 2894, 28...",NaN,none 1 0 0 1 1,1,0,0,0
4,f6293b1c49e2,00292f8c37bd,M,"[0.139, 0.139]",MONOCHROME2,2539,3050,"[[86, 88, 86, 86, 87, 86, 87, 87, 87, 87, 87, ...",NaN,none 1 0 0 1 1,1,0,0,0


In [11]:
X = big_df['pixel_array'] # ??? à vérifier, à passer en np.array ou en matrice? garder plus d'infos dans le df ?
y = big_df[['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']]

In [12]:
print(type(X))
print(X.shape)
len(X)

<class 'pandas.core.series.Series'>
(5,)


5

## Modèle CNN

In [ ]:
#train_val_test_split
